In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import time
import sys
import os
import pandas as pd
import mne
import matplotlib
from datetime import datetime
from mne_icalabel import label_components

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

root = 'F:/Documents/Science/MirRevAdaptEEG'

# Set the events
event_dict = {
        #trial triggers        
        "trial_onset": 16138,
        "target_onset": 16139,
        "go_target_onset": 16140,
        "reach_onset": 16141,
        "reach_end_feedback_onset": 16142, #reach end is when step moves to 5, which is also onset for feedback
        "feedback_home": 16143,
        "fixation_onset": 16144,
        #task triggers
        "task_rest": 16148,
        "task_aligned": 16149,
        "task_random0": 16150,
        "task_rotation": 16151,
        "task_washout0": 16152,
        "task_random1": 16153,
        "task_mirror": 16154,
        "task_washout1": 16155,
        "task_end": 16156,
        #target locations based on participant condition (only get 12 of below)
        "target_q1_hor_near": 16158,
        "target_q1_hor_mid": 16159,
        "target_q1_hor_far": 16160,
        "target_q3_hor_near": 16161,
        "target_q3_hor_mid": 16162,
        "target_q3_hor_far": 16163,
        "target_q1_ver_near": 16164,
        "target_q1_ver_mid": 16165,
        "target_q1_ver_far": 16166,
        "target_q3_ver_near": 16167,
        "target_q3_ver_mid": 16168,
        "target_q3_ver_far": 16169,

        "target_q2_hor_near": 16178,
        "target_q2_hor_mid": 16179,
        "target_q2_hor_far": 16180,
        "target_q4_hor_near": 16181,
        "target_q4_hor_mid": 16182,
        "target_q4_hor_far": 16183,
        "target_q2_ver_near": 16184,
        "target_q2_ver_mid": 16185,
        "target_q2_ver_far": 16186,
        "target_q4_ver_near": 16187,
        "target_q4_ver_mid": 16188,
        "target_q4_ver_far": 16189,

        #extra target info triggers come up in recording
        "targetinfo_1": 16175,
        "targetinfo_2": 16190,
        "targetinfo_3": 16191
}

In [2]:
#setting up path/ directory

#access specific file for specific participant
def load_bdf(pp_num, root_dir):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    pp_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0.bdf' % pp_num)

    data = mne.io.read_raw_bdf(filename, preload = True, exclude = ['EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8'])
    return data, pp_directory

In [12]:
def set_reference(pp_num, root_dir):
    
    data, pp_directory = load_bdf(pp_num, root_dir)
    
    #use mastoids as reference
    data_ref = mne.io.Raw.set_eeg_reference(data, ref_channels = ['EXG1', 'EXG2'])

    #drop reference electrodes from further analyses
    data_ref.drop_channels(['EXG1', 'EXG2'])

    #set montage
    data_ref.set_montage('biosemi64') #only for plotting, can also be "standard_1020"
    
    return data_ref

In [13]:
#set filter parameters

def apply_filter(pp_num, root_dir, hcutoff, lcutoff, linefr):
    
    data = set_reference(pp_num, root_dir)
    
    line_freqs = linefr

    #highpass at 0.1 removes slow wave drift
    data_filt = data.filter(l_freq = hcutoff, h_freq= lcutoff)
    data_filt = data_filt.notch_filter(line_freqs, filter_length='auto', phase='zero')

    # plot density to see after filtering
    # raw_filt.compute_psd(fmax=250).plot()
    
    return data_filt

In [ ]:
# # Epoch using target information (same as trial start)
# # First epoch using event_dict. This will create an Epoch around all triggers in event_dict.
# def get_epochs(data):

#     targetinfo = ["target_q1_hor_near","target_q1_hor_mid","target_q1_hor_far","target_q3_hor_near","target_q3_hor_mid","target_q3_hor_far","target_q1_ver_near","target_q1_ver_mid","target_q1_ver_far","target_q3_ver_near","target_q3_ver_mid","target_q3_ver_far","target_q2_hor_near","target_q2_hor_mid","target_q2_hor_far","target_q4_hor_near","target_q4_hor_mid","target_q4_hor_far","target_q2_ver_near","target_q2_ver_mid","target_q2_ver_far","target_q4_ver_near","target_q4_ver_mid","target_q4_ver_far","targetinfo_1","targetinfo_2","targetinfo_3"]
#     event_values = [event_dict[i] for i in targetinfo]
#     epochs = mne.Epochs(data, events, event_id = event_values, on_missing = 'ignore', baseline = None, tmin=-2.5, tmax=10, preload = True)
#     return epochs

In [10]:
# Epoch using feedback onset
# First epoch using event_dict. This will create an Epoch around all triggers in event_dict.
def get_epochs(pp_num, root_dir, hpass, lpass, linefreqs, trigger):
    
    data = apply_filter(pp_num, root_dir, hpass, lpass, linefreqs)
    
    events = mne.find_events(data, initial_event=False)
    
    #marker = ["target_q1_hor_near","target_q1_hor_mid","target_q1_hor_far","target_q3_hor_near","target_q3_hor_mid","target_q3_hor_far","target_q1_ver_near","target_q1_ver_mid","target_q1_ver_far","target_q3_ver_near","target_q3_ver_mid","target_q3_ver_far","target_q2_hor_near","target_q2_hor_mid","target_q2_hor_far","target_q4_hor_near","target_q4_hor_mid","target_q4_hor_far","target_q2_ver_near","target_q2_ver_mid","target_q2_ver_far","target_q4_ver_near","target_q4_ver_mid","target_q4_ver_far","targetinfo_1","targetinfo_2","targetinfo_3"]
    marker = [trigger]
    event_values = [event_dict[i] for i in marker]
    
    epochs = mne.Epochs(data, events, event_id = event_values, on_missing = 'ignore', baseline = None, tmin=-1.5, tmax=1.5, preload = True)
    return epochs

In [ ]:
def fit_ICA(data, plot = False):
    # Plot is true or false
    # set up and fit the ICA
    
    ica = mne.preprocessing.ICA(random_state=97, max_iter=1000, method = 'infomax', fit_params=dict(extended=True))
    ica.fit(data)
    
    #ICLabel to get automatic labels for components
    iclabel = label_components(data, ica, method='iclabel')
    
    if plot == True:
        ica.plot_sources(data, show_scrollbars=True, start = 0, stop = 1)
    
    return ica, iclabel

In [ ]:
def apply_ICA(threshold, iclabel, data_to_apply, ica, plot = False):

    # Convert ICLabel results to data frame
    IC_df = pd.DataFrame(data=iclabel)
    # Get indices for ICs to remove
    IC_exclude_idx = IC_df[(IC_df.y_pred_proba>threshold)&(IC_df.labels!='other')&(IC_df.labels!='brain')].index.values

    # Mark the selected ICs for exclusion
    ica.exclude = IC_exclude_idx
    
    # Apply the ICA solution to the original data (filtered at 0.1Hz) and save the solution to examine later.
    ica.apply(data_to_apply)
    
    if plot == True:
    # Plot the ICA-excluded Epoched data
        data_to_fit.plot(events = events, n_epochs = 1, event_id = event_dict, event_color = 'red')
    
    return data_to_apply, IC_df

In [ ]:
def save_ICA(pp_num, IC_df, ica, ppdir, potential):
    patherp = os.path.join(ppdir, potential)
    is_exist = os.path.exists(patherp)
    if not is_exist:
        os.makedirs(patherp)
    
    # Save the ICA results 
    ica_filename = os.path.join(patherp, 'mra_p%03d_run0-ica.fif' % pp_num)
    ica.save(ica_filename, overwrite = True)

    # Save the iclabel output
    df_filename = os.path.join(patherp, 'mra_p%03d_run0_IClabel.csv' % pp_num)
    IC_df.to_csv(df_filename)

# Uncomment if you want to read ica data back in.
# read_ica = mne.preprocessing.read_ica(ica_filename)

In [ ]:
def downsample(data_epoch, freq):

    epochs_ds = data_epoch.resample(sfreq=freq)
    #epochs_ds

    #epochs_ds.plot(events=events, event_color='red')
    #epochs_ds.compute_psd(fmax=50).plot()
    
    return epochs_ds

In [ ]:
def output_epochs(pp_num, data, ppdir, potential):
    patherp = os.path.join(ppdir, potential)
    is_exist = os.path.exists(patherp)
    if not is_exist:
        os.makedirs(patherp)
    # Save the cleaned data :)
    out_fname = os.path.join(patherp, 'mra_p%03d_run0-epo.fif' % pp_num)
    data.save(out_fname, overwrite = True)

In [ ]:
def prepro_data(pp = range(0,32), linefreqs = (60, 120, 180, 240), hpass = 0.1, hpassICA = 1, lpass = 200, trigger = "reach_onset", potential = "lrp", exc_prob = .8, dsfreq = 200):

    for pp_idx in pp:
        raw, pp_directory = load_bdf(pp_idx, root)
        epoch_data = get_epochs(pp_idx, root, hpass, lpass, linefreqs, trigger)
        epoch_ICA = get_epochs(pp_idx, root, hpassICA, lpass, linefreqs, trigger)
    
        ica_dat, labels = fit_ICA(epoch_ICA)
        epochs, IC_export = apply_ICA(exc_prob, labels, epoch_data, ica_dat)
        save_ICA(pp_idx, IC_export, ica_dat, pp_directory, potential)
        epochs_ds = downsample(epochs, dsfreq)
        output_epochs(pp_idx, epochs_ds, pp_directory, potential)
        
        #take note of time when one participant finishes
        now = datetime.now()
        timestamp_format = "%Y-%m-%d %H-%M"
        timestamp_str = now.strftime(timestamp_format)
        
        patherp = os.path.join(pp_directory, potential)
        with open( os.path.join(patherp, 'mra_p%03d_run0-time.txt' % pp_idx), "w") as file:
            file.write(timestamp_str)
    

In [ ]:
prepro_data()

In [15]:
pp_idx = 0
root = 'F:/Documents/Science/MirRevAdaptEEG'
raw, pp_directory = load_bdf(pp_idx, root)
epoch_data = get_epochs(pp_idx, root, hpass=0.1, lpass=200, linefreqs = (60, 120, 180, 240), trigger = 'reach_onset')
epoch_data

Extracting EDF parameters from F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\mra_p000_run0.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1561087  =      0.000 ...  3048.998 secs...
Extracting EDF parameters from F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\mra_p000_run0.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1561087  =      0.000 ...  3048.998 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper pas

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)



[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Trigger channel has a non-zero initial value of 81664 (consider using initial_event=True to detect this event)
3000 events found
Event IDs: [16138 16139 16140 16141 16142 16143 16144 16148 16149 16150 16151 16152
 16153 16154 16155 16156 16158 16159 16160 16161 16162 16163 16164 16165
 16166 16167 16168 16169 81664]


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    3.1s finished


Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 420 events and 1537 original time points ...
0 bad epochs dropped


Number of events,420
Events,16141: 420
Time range,-1.500 – 1.500 sec
Baseline,off


In [20]:
epoch_data.plot()

C:\Users\Raphael\anaconda3\envs\mne\lib\site-packages\torch\distributed\distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


In [16]:
datdf=epoch_data.to_data_frame()
datdf

,time,condition,epoch,Fp1,AF7,AF3,F1,F3,F5,F7,...,CP2,P2,P4,P6,P8,P10,PO8,PO4,O2,Status
0,-1.500000,16141,9,-76.331174,-44.477178,-46.527339,2.001183,-24.299148,-15.740877,-21.978175,...,-12.662492,-21.942047,-4.627354,-17.260346,-20.361646,-34.659917,-25.878017,-23.232025,-35.155024,16128.0
1,-1.498047,16141,9,-75.837105,-44.265686,-47.462951,3.233162,-23.736762,-16.495385,-21.607540,...,-11.003379,-20.631763,-1.341333,-13.508022,-14.381541,-29.071412,-18.834867,-20.063850,-29.592602,16128.0
2,-1.496094,16141,9,-75.036998,-44.016030,-47.107277,3.870279,-22.875859,-15.857944,-21.772074,...,-7.285086,-16.606371,4.046359,-5.100832,-2.692135,-25.707904,-8.931597,-13.948234,-23.783258,16128.0
3,-1.494141,16141,9,-78.124833,-46.177776,-47.303055,3.287144,-24.742230,-16.801722,-23.783293,...,-5.249371,-14.782690,4.964580,-2.777899,1.311039,-21.964316,-8.952827,-12.362052,-20.466374,16128.0
4,-1.492188,16141,9,-87.727585,-55.739606,-55.272065,-4.581510,-32.739362,-24.624391,-31.627219,...,-9.894388,-18.420764,-0.770809,-10.133386,-7.615582,-22.242602,-13.302378,-16.911975,-22.948323,16128.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645535,1.492188,16141,2992,-31.960650,-18.242407,-10.026307,19.444046,-0.277615,-18.426051,-5.357766,...,14.892075,0.502496,12.793704,1.346702,1.836240,8.512260,-3.179852,5.714684,11.765377,16128.0
645536,1.494141,16141,2992,-37.163635,-26.693975,-8.501183,18.660271,-0.816152,-17.318636,-7.433674,...,11.997005,-2.757020,9.505201,-2.294399,-3.069971,6.409110,-6.152741,3.178472,9.752104,16128.0
645537,1.496094,16141,2992,-41.276470,-32.215684,-14.397869,16.868018,-1.215408,-18.774479,-11.643306,...,9.059676,-5.940699,7.189688,-3.835948,-5.109537,5.276927,-6.191804,1.435062,9.210372,16128.0
645538,1.498047,16141,2992,-40.418540,-32.881042,-18.597856,16.826813,1.624750,-18.598931,-12.593425,...,7.372318,-8.096262,7.054333,-3.604732,-3.431595,7.288285,-5.699702,0.709369,8.816403,16128.0


In [8]:
datdf.iloc[0:11,:]

,time,Fp1,AF7,AF3,F1,F3,F5,F7,FT7,FC5,...,P4,P6,P8,P10,PO8,PO4,O2,EXG1,EXG2,Status
0,0.000000,-12631.898535,-2405.448681,-7429.626898,4218.007830,-3886.820944,-2134.730431,1297.169477,2534.760940,10063.965778,...,8912.717906,4994.818895,-4061.008122,206.077743,3691.477553,8809.061847,14856.644423,5254.287165,25572.937122,81664.0
1,0.001953,-12642.679765,-2411.292420,-7433.564391,4215.757834,-3888.102192,-2133.292934,1297.075727,2533.948442,10068.559520,...,8914.561652,4995.475144,-4063.445618,205.983993,3690.977554,8812.093092,14857.644421,5259.412156,25573.187121,81664.0
2,0.003906,-12656.367240,-2422.104900,-7439.376880,4214.257837,-3889.508439,-2133.886683,1292.044487,2531.573446,10069.778268,...,8915.311651,4993.537647,-4066.851861,205.765244,3689.946306,8812.155591,14855.675675,5263.568398,25566.655883,81664.0
3,0.005859,-12661.117231,-2425.417394,-7440.501878,4215.476585,-3887.977192,-2134.199182,1291.638237,2531.354697,10073.840760,...,8917.905396,4994.568895,-4064.789365,208.296489,3691.321303,8813.530589,14856.800672,5252.193419,25562.749641,81664.0
4,0.007812,-12658.242237,-2422.417400,-7436.783135,4218.789079,-3883.852200,-2133.011685,1297.325727,2537.698435,10077.528253,...,8920.749141,4999.318887,-4057.945628,215.921475,3696.102545,8816.061834,14859.738167,5248.443426,25561.374643,81664.0
5,0.009766,-12662.617229,-2427.261141,-7439.408130,4217.539081,-3885.508446,-2136.011679,1294.231983,2534.479691,10075.559507,...,8920.405391,5002.350131,-4053.883135,219.733968,3699.633788,8816.968083,14861.956913,5250.599672,25556.968401,81664.0
6,0.011719,-12669.617216,-2434.292378,-7442.564374,4215.664085,-3885.914696,-2136.636678,1290.825739,2525.917207,10085.653238,...,8917.842896,5000.537634,-4055.039383,215.483976,3699.165039,8815.436835,14861.331914,5250.755922,25558.718398,81664.0
7,0.013672,-12677.523451,-2439.698618,-7445.501869,4214.945336,-3884.820948,-2135.074181,1291.575738,2521.667215,10088.153234,...,8916.686648,4998.006389,-4056.320631,212.952731,3697.321292,8812.655591,14859.144418,5254.630915,25569.968377,81664.0
8,0.015625,-12691.335926,-2452.261095,-7455.751850,4206.414102,-3894.227180,-2142.636667,1286.513247,2531.635946,10058.840788,...,8915.530400,4994.912645,-4060.570623,210.890234,3693.290050,8810.686844,14856.456923,5252.349669,25570.812126,81664.0
9,0.017578,-12700.523409,-2457.167336,-7461.158090,4201.164111,-3899.383421,-2142.292918,1279.857009,2536.792187,10062.653281,...,8917.905396,4996.193892,-4061.664371,213.390230,3695.196296,8814.780587,14860.113166,5252.068420,25574.030870,81664.0
